In [239]:
import numpy as np
from sympy import symbols, And, Not, simplify, solve, reduce_inequalities
%run constants.ipynb
%run li_handler.ipynb

In [2]:
def no_loser(a,b,c):
    return [simplify(i) for i in [a>0, b>0, c>0]]

In [3]:
def parentStates(a,b,c, r=V):
    return [s for s in 
            [(2*a,b-a,c), (2*a,b,c-a),
            (a-b,2*b,c), (a,2*b,c-b),
            (a-c,b,2*c), (a,b-c,2*c)] 
            if feasible(*no_loser(*s), r=r)]

In [4]:
@memoized
def coef(n):
    return Fraction(1, 6**n)

In [5]:
def concat(ls):
    ''' Flatten 2D list. '''
    return [j for l in ls for j in l]

In [6]:
import csv
from itertools import islice

def read(filename, min_n=1, max_n=5):
    hs = {}
    with open(filename, 'r') as fp:
        reader = islice(csv.reader(fp), min_n-1, max_n)
        for row in reader:
            n, c, rs = row
            # parse entries
            hs[int(n)] = (int(c), sympify(rs))
    return hs

In [7]:
def h_mult(n,a,b,c):
    return non_redundant(*no_loser(a,b,c), r=V)

def mult_inds(inds, prereqs, r=V):
    '''
    prereqs * sum(inds), keeping only non-redundant indicators in `prereqs` given restriction `r`.
    '''
    return [i + [p for p in non_redundant(*prereqs, r=V+i)] for i in inds]

@memoized
def _h(n,a,b,c):
    ''' Returns the additive indicators of h_n(a,b,c), as a 2D list. '''
    # constant coef = (1/6)^n
    # so we only keep track of the indicators to be summed up
    if n == 1:
        return mult_inds([[simplify(a<=b)], [simplify(a<=c)]], h_mult(1,a,b,c))
    # list of inequalities representing the region Rn
    return mult_inds(concat([_h(n-1, *s) for s in parentStates(a,b,c)]), 
                     h_mult(n,a,b,c))

@memoized
def h(n,a,b,c, regenerate):
    if regenerate:
        return reduce(_h(n,a,b,c), r=V)
    else:
        if (a,b,c) == (y,x,z):
            return read(H_YXZ_CACHE, max_n=n)[n]
        else:
            return read(H_XYZ_CACHE, max_n=n)[n]

# General constraints

In [68]:
n = 3
H_XYZ = read(H_XYZ_CACHE, min_n=1, max_n=n)
H_YXZ = read(H_YXZ_CACHE, min_n=1, max_n=n)
num_pos_per_n = [len(H_XYZ[i][1]) for i in range(1,n+1)]
num_neg_per_n = [len(H_YXZ[i][1]) for i in range(1,n+1)]
num_pos_per_n

[0, 5, 31]

In [69]:
num_pos = sum(num_pos_per_n)  # number of positive compound indicators
num_neg = sum(num_neg_per_n)
num_pos

36

In [70]:
pos_coefs = concat([repeat(j, k) for j,k in zip([coef(i) for i in range(1,n+1)], num_pos_per_n)])
neg_coefs = concat([repeat(j, k) for j,k in zip([-coef(i) for i in range(1,n+1)], num_neg_per_n)])

In [71]:
const = sum([coef(i)*(H_XYZ[i][0] - H_YXZ[i][0]) for i in range(1,n+1)])
const

Fraction(1, 6)

In [72]:
all_pos_comp_inds = concat([H_XYZ[i][1] for i in range(1,n+1)])
all_neg_comp_inds = concat([H_YXZ[i][1] for i in range(1,n+1)])
all_pos_comp_inds

[[3*x <= y],
 [2*x <= z],
 [2*x <= y],
 [3*x <= z],
 [x <= -y + z],
 [7*x <= y],
 [4*x <= z, 3*x < y],
 [5*x <= y],
 [6*x <= z],
 [5*x <= 3*y, y < 3*x],
 [3*x <= y + z, y < 3*x],
 [2*x <= y],
 [x <= -y + z],
 [3*x <= y + z, z < 2*x],
 [z < 2*x],
 [6*x <= y],
 [5*x <= z, 2*x < y],
 [4*x <= y],
 [7*x <= z],
 [y < 2*x],
 [3*x <= y + z, y < 2*x],
 [x < -y + z],
 [3*x <= -y + z],
 [3*x <= y + z, z < 3*x],
 [5*x <= 3*z, z < 3*x],
 [2*x <= z, -y + z < x],
 [3*x <= 2*y],
 [2*x <= -y + z],
 [x < -y + z],
 [3*x <= -y + z],
 [3*y < z],
 [x <= -3*y + z],
 [-y + z < x],
 [x <= -3*y + 3*z, -y + z < x],
 [x <= 3*y - z],
 [x <= -2*y + 2*z, z < 3*y]]

In [73]:
model = gp.Model(f'f_{n}(x,y,z)-thresh{n}', env=gp.Env())
model.Params.OutputFlag = 1
model.Params.LogToConsole = 1
model.Params.MIPFocus = 1
# model.Params.CliqueCuts = 1
# model.Params.FeasibilityTol = 1e-9
# model.Params.IntFeasTol = 1e-9

gx = model.addVar(lb=1, vtype='I', name='x')
gy = model.addVar(lb=2, vtype='I', name='y')
gz = model.addVar(lb=3, vtype='I', name='z')

# gx = model.addVar(lb=1, name='x')
# gy = model.addVar(lb=2, name='y')
# gz = model.addVar(lb=3, name='z')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Set parameter MIPFocus to value 1


In [76]:
def add_ind_constr(ineq, model=model, name='ind', var_map=SP_TO_GP):
    ''' Add indicator constraint for a single inequality'''
    r = unify_ineq(ineq)  # to the form >= 0
    lhs = to_gp_linexpr(r.lhs, var_map=var_map)
    v = model.addVar(vtype=GRB.BINARY, name=name)
    model.addConstr((v == 1) >> (lhs >= 0))
    model.addConstr((v == 0) >> (lhs <= -EPS))  # lhs < 0
    return v

In [77]:
# for each compound indicator group, restrict their sum to be [0,1]
def add_ind_constrs(*ineqs, model=model, var_map=SP_TO_GP):
    ''' Adds variables and constraints for a compound indicator. '''
    if len(ineqs) == 1:
        return (add_ind_constr(ineqs[0], 
                               name=var_name(ineqs[0], prefix='single_ind'), 
                               var_map=var_map,
                               model=model),)
    
    I = model.addVar(vtype=GRB.BINARY, 
                     name=var_name(*ineqs, prefix='main_ind'))
    component_vars = [add_ind_constr(i, 
                                     name=var_name(i, prefix='component_ind'), 
                                     var_map=var_map,
                                     model=model) for i in ineqs]
    model.addConstr(I == and_(component_vars), 
                    name=var_name(*ineqs, prefix='andconstr'))
    return I, component_vars

In [78]:
pos_inds = [add_ind_constrs(*i) for i in all_pos_comp_inds]
neg_inds = [add_ind_constrs(*i) for i in all_neg_comp_inds]
pos_inds

[(<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),


In [79]:
obj = gp.quicksum([pos_coefs[i]*pos_inds[i][0] for i in range(num_pos)]) + \
        gp.quicksum([neg_coefs[i]*neg_inds[i][0] for i in range(num_neg)]) + const - thresh(n)

In [465]:
model.setObjective(obj, GRB.MINIMIZE)
f_constr = model.addConstr(obj <= 0, name='non_pos_f')
# constraints for V
model.addConstr(gx <= gy - 1, name='V1')
model.addConstr(gy <= gz - 1, name='V2')

<gurobi.Constr *Awaiting Model Update*>

In [466]:
model.optimize(my_callback)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 111 columns and 70 nonzeros
Model fingerprint: 0xb8e966f6
Model has 195 general constraints
Variable types: 0 continuous, 111 integer (108 binary)
Coefficient statistics:
  Matrix range     [5e-03, 1e+00]
  Objective range  [5e-03, 3e-02]
  Bounds range     [1e+00, 3e+00]
  RHS range        [7e-02, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 7e+00]
Presolve added 292 rows and 261 columns
Presolve time: 0.01s
Presolved: 295 rows, 372 columns, 907 nonzeros
Presolved model has 174 SOS constraint(s)
Variable types: 0 continuous, 372 integer (195 binary)

Root relaxation: objective -1.648148e-01, 158 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | In

In [377]:
model.MIPGap

0.0

In [28]:
def analyse_result(model):
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found:")
        for v in model.getVars():
            print(f"{v.varName}: {v.x}, {not (not v.x)}")
        print(f"Objective value: {model.objVal}")
    elif model.status == GRB.UNBOUNDED:
        print('The model cannot be solved because it is unbounded')
    elif model.status != GRB.INF_OR_UNBD and model.status != GRB.INFEASIBLE:
        print('Optimization was stopped with status %d' % model.status)
    else:
        # Relax the bounds and try to make the model feasible
        print('The model is infeasible; relaxing the bounds')
        # can only relax thresh constraint
        relaxable_vars = [gx, gy, gz]
        pen = [1] * len(relaxable_vars)
        model.feasRelax(1, False, 
                        relaxable_vars, pen, pen, 
                        [f_constr], 
                        [1])
        model.optimize()

In [342]:
analyse_result()

Optimal solution found:
x: 4.0, True
y: 5.0, True
z: 6.0, True
single_ind_3*x <= y: -0.0, False
single_ind_2*x <= z: -0.0, False
single_ind_2*x <= y: -0.0, False
single_ind_3*x <= z: -0.0, False
single_ind_x <= -y + z: -0.0, False
single_ind_7*x <= y: -0.0, False
main_ind_(4*x <= z, 3*x < y): -0.0, False
component_ind_4*x <= z: -0.0, False
component_ind_3*x < y: -0.0, False
single_ind_5*x <= y: -0.0, False
single_ind_6*x <= z: -0.0, False
main_ind_(5*x <= 3*y, y < 3*x): -0.0, False
component_ind_5*x <= 3*y: -0.0, False
component_ind_y < 3*x: 1.0, True
main_ind_(3*x <= y + z, y < 3*x): -0.0, False
component_ind_3*x <= y + z: -0.0, False
component_ind_y < 3*x: 1.0, True
single_ind_2*x <= y: -0.0, False
single_ind_x <= -y + z: -0.0, False
main_ind_(3*x <= y + z, z < 2*x): -0.0, False
component_ind_3*x <= y + z: -0.0, False
component_ind_z < 2*x: 1.0, True
single_ind_z < 2*x: 1.0, True
single_ind_6*x <= y: -0.0, False
main_ind_(5*x <= z, 2*x < y): -0.0, False
component_ind_5*x <= z: -0.0, 

In [240]:
def eval_f(n,a,b,c):
    tot = 0.
    vals = {x:a, y:b, z:c}
    for i in range(1,n+1):
        pos_c, pos_rs = H_XYZ[i]
        neg_c, neg_rs = H_YXZ[i]
        tot += coef(i) * (pos_c - neg_c + sum([sub(r, vals) for r in pos_rs]) - sum([sub(r, vals) for r in neg_rs]))
    return tot - thresh(n)

In [241]:
eval_f(3,5,6,10)

0.04351851851851851

In [203]:
model.write('modelB.lp')

In [110]:
def mip(n, output_file=None, print_result=True):
    failed_regions = []
    
    def extract_region(model):
        region = []
        variables = model.getVars()
        sol = model.cbGetSolution(variables)
        for v, val in zip(variables, sol):
            # only look at the individual indicators
            if not 'single_ind' in v.varName and not 'component_ind' in v.varName:
                continue
            e = extract_ineq(v)
            if val == 1 and e is not None:
                region.append(e[0])
            elif val == 0:
                # indicator is false
                # guaranteed single inequality
                region.append(Not(e[0]))
        return region
    
    def pos_f_callback(model, where):
        if where == GRB.Callback.MIPSOL:
            # Capture node information
            try:
                obj = model.cbGet(GRB.Callback.MIPSOL_OBJ)
                print("Current objective:", obj)
                if obj > 0:  # Node is infeasible
                    failed_regions.append(extract_region(model))
            except gp.GurobiError as e:
                print(f"Gurobi error during callback: {e}")
            except Exception as e:
                print(f"Unexpected error during callback: {e}")
    
    h_xyz = read(H_XYZ_CACHE, min_n=1, max_n=n)
    h_yxz = read(H_YXZ_CACHE, min_n=1, max_n=n)
    num_pos_per_n = [len(H_XYZ[i][1]) for i in range(1,n+1)]
    num_neg_per_n = [len(H_YXZ[i][1]) for i in range(1,n+1)]
    num_pos = sum(num_pos_per_n)
    num_neg = sum(num_neg_per_n)
    
    pos_coefs = concat([repeat(j, k) for j,k in zip([coef(i) for i in range(1,n+1)], num_pos_per_n)])
    neg_coefs = concat([repeat(j, k) for j,k in zip([-coef(i) for i in range(1,n+1)], num_neg_per_n)])
    const = sum([coef(i)*(H_XYZ[i][0] - H_YXZ[i][0]) for i in range(1,n+1)])
    
    all_pos_comp_inds = concat([H_XYZ[i][1] for i in range(1,n+1)])
    all_neg_comp_inds = concat([H_YXZ[i][1] for i in range(1,n+1)])
    
    model = gp.Model(f'f_{n}(x,y,z)-thresh{n}', env=gp.Env())
    model.Params.OutputFlag = 1
    model.Params.LogToConsole = 1
    model.Params.MIPFocus = 1
    
    gx = model.addVar(lb=1, vtype='I', name='x')
    gy = model.addVar(lb=2, vtype='I', name='y')
    gz = model.addVar(lb=3, vtype='I', name='z')
    sp_to_gp = {x: gx, y: gy, z: gz}
    
    pos_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp) \
                for i in all_pos_comp_inds]
    neg_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp) \
                for i in all_neg_comp_inds]
    
    obj = gp.quicksum([pos_coefs[i]*pos_inds[i][0] for i in range(num_pos)]) + \
        gp.quicksum([neg_coefs[i]*neg_inds[i][0] for i in range(num_neg)]) + const - thresh(n)
    
    model.setObjective(obj, GRB.MINIMIZE)
#     f_constr = model.addConstr(obj <= 0, name='non_pos_f')
    # constraints for V
    model.addConstr(gx <= gy - 1, name='V1')
    model.addConstr(gy <= gz - 1, name='V2')
    
    model.optimize(pos_f_callback)
    
    if print_result:
        print(analyse_result(model))
    
    if output_file:
        model.write(output_file)
    
    return (model.status == GRB.OPTIMAL and \
            model.MIPGap == 0 and \
            eval_f(n, gx.x, gy.x, gz.x) < 0), \
           model, \
           failed_regions

In [111]:
status, model, failed_regions = mip(3, print_result=False)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Set parameter MIPFocus to value 1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 111 columns and 4 nonzeros
Model fingerprint: 0x4040817d
Model has 195 general constraints
Variable types: 0 continuous, 111 integer (108 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-03, 3e-02]
  Bounds range     [1e+00, 3e+00]
  RHS range        [1e+00, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 7e+00]
Presolve added 292 rows and 261 columns
Presolve time: 0.01s
Presolved: 294 rows, 372 columns, 841 nonzeros
Presolved model has 174 SOS constraint(s)
Variable types: 0 continuous, 372 integer (195 binary)
Current objective: 0.10370370370370363
Found heuristic solution: objective 0.1

In [112]:
failed_regions

[[3*x > y,
  2*x <= z,
  2*x <= y,
  3*x <= z,
  x <= -y + z,
  7*x > y,
  4*x > z,
  3*x >= y,
  5*x > y,
  6*x > z,
  5*x <= 3*y,
  y < 3*x,
  3*x <= y + z,
  y < 3*x,
  2*x <= y,
  x <= -y + z,
  3*x <= y + z,
  z >= 2*x,
  z >= 2*x,
  6*x > y,
  5*x > z,
  2*x >= y,
  4*x > y,
  7*x > z,
  y >= 2*x,
  3*x <= y + z,
  y >= 2*x,
  x >= -y + z,
  3*x > -y + z,
  3*x <= y + z,
  z >= 3*x,
  5*x <= 3*z,
  z >= 3*x,
  2*x <= z,
  -y + z >= x,
  3*x <= 2*y,
  2*x > -y + z,
  x >= -y + z,
  3*x > -y + z,
  3*y >= z,
  x > -3*y + z,
  -y + z >= x,
  x <= -3*y + 3*z,
  -y + z >= x,
  x <= 3*y - z,
  x <= -2*y + 2*z,
  z < 3*y,
  3*y > z,
  y <= 3*x,
  y <= 2*x,
  x <= -y + z,
  7*y > z,
  2*y > 3*x,
  3*y - z > x,
  x > -3*y + z,
  3*y - z > x,
  z < 3*y,
  5*y > 3*z,
  z < 3*y,
  3*y > 5*x,
  2*y - z <= 2*x,
  y < 3*x,
  y <= 2*x,
  3*y - z <= 3*x,
  y <= 7*x,
  3*x >= y,
  3*x >= y,
  y <= 5*x,
  2*x < z,
  x <= -y + z,
  y + z > 3*x,
  z >= 2*x,
  x > -2*y + z,
  y >= 2*x,
  x > -3*y + z,

## Plot region

In [144]:
import numpy as np
import matplotlib.pyplot as plt

In [222]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

XYZ_SUM = 3000

# Define the inequalities
inequalities = failed_regions[4]

# Combine all inequalities
all_ineqs = inequalities + V

# Convert inequalities to functions for evaluation
ineq_funcs = [sp.lambdify((x, y), ineq.subs(z, XYZ_SUM - x - y), 'numpy') for ineq in all_ineqs]

# Define ranges for x and y based on the constraints
x_vals = np.linspace(1, XYZ_SUM // 3, 500)  # Avoid x = 0
y_vals = np.linspace(1, XYZ_SUM // 2, 500)  # Avoid y = 0

# Create meshgrid for x and y
X, Y = np.meshgrid(x_vals, y_vals)
Z = XYZ_SUM - X - Y  # Corresponding z values based on x and y

# Initialize the valid region as all True
valid_region = np.ones_like(X, dtype=bool)

# Evaluate each inequality over the grid
for func in ineq_funcs:
    valid_region &= func(X, Y)

# Plotting the shaded region
plt.figure(figsize=(10, 8))
plt.xlabel('x')
plt.ylabel('y')
plt.title('Shaded Region Bounded by Inequalities: 3x + y <= z, 5y > z, x + y + z = 1000, 0 < x < y < z')

# Shade the feasible and valid region
plt.contourf(X, Y, valid_region, levels=[0.5, 1], colors=['skyblue'], alpha=0.6)

plt.xlim([0, XYZ_SUM // 3])
plt.ylim([0, XYZ_SUM // 2])
plt.grid(True)
plt.show()


IndexError: list index out of range

In [208]:
XYZ_SUM = 1000
for 
failed_eqs = [solve(solve(r,y).subs(z, XYZ_SUM-x-y),y) for rs in failed_regions for r in rs]

[Eq(3*x, y),
 Eq(2*x, -x - y + 1000),
 Eq(2*x, y),
 Eq(3*x, -x - y + 1000),
 Eq(x, -x - 2*y + 1000),
 Eq(7*x, y),
 Eq(4*x, -x - y + 1000),
 Eq(3*x, y),
 Eq(5*x, y),
 Eq(6*x, -x - y + 1000),
 Eq(5*x, 3*y),
 Eq(y, 3*x),
 Eq(3*x, 1000 - x),
 Eq(y, 3*x),
 Eq(2*x, y),
 Eq(x, -x - 2*y + 1000),
 Eq(3*x, 1000 - x),
 Eq(-x - y + 1000, 2*x),
 Eq(-x - y + 1000, 2*x),
 Eq(6*x, y),
 Eq(5*x, -x - y + 1000),
 Eq(2*x, y),
 Eq(4*x, y),
 Eq(7*x, -x - y + 1000),
 Eq(y, 2*x),
 Eq(3*x, 1000 - x),
 Eq(y, 2*x),
 Eq(x, -x - 2*y + 1000),
 Eq(3*x, -x - 2*y + 1000),
 Eq(3*x, 1000 - x),
 Eq(-x - y + 1000, 3*x),
 Eq(5*x, -3*x - 3*y + 3000),
 Eq(-x - y + 1000, 3*x),
 Eq(2*x, -x - y + 1000),
 Eq(-x - 2*y + 1000, x),
 Eq(3*x, 2*y),
 Eq(2*x, -x - 2*y + 1000),
 Eq(x, -x - 2*y + 1000),
 Eq(3*x, -x - 2*y + 1000),
 Eq(3*y, -x - y + 1000),
 Eq(x, -x - 4*y + 1000),
 Eq(-x - 2*y + 1000, x),
 Eq(x, -3*x - 6*y + 3000),
 Eq(-x - 2*y + 1000, x),
 Eq(x, x + 4*y - 1000),
 Eq(x, -2*x - 4*y + 2000),
 Eq(-x - y + 1000, 3*y),
 Eq(3*y,

In [199]:
def apply_constr(ineqs, xs, ys):
    ineq_funcs = [sp.lambdify((x, y), ineq.subs(z, XYZ_SUM - x - y), 'numpy') for ineq in (ineqs + V)]
    X, Y = np.meshgrid(xs, ys)
    Z = XYZ_SUM - X - Y
    valid_region = np.ones_like(X, dtype=bool)
    
#     for i, func in zip(ineqs + V, ineq_funcs):
    for func in ineq_funcs:
        valid_region &= func(X, Y)
#         if any([any(row) for row in valid_region]):
#             continue
#         else:
#             print("stop at:", i)
#             break
    
    return X, Y, valid_region

In [198]:
apply_constr(failed_regions[2], x_vals, y_vals)

(array([[  1,   1,   2, ..., 997, 998, 999],
        [  1,   1,   2, ..., 997, 998, 999],
        [  1,   1,   2, ..., 997, 998, 999],
        ...,
        [  1,   1,   2, ..., 997, 998, 999],
        [  1,   1,   2, ..., 997, 998, 999],
        [  1,   1,   2, ..., 997, 998, 999]]),
 array([[  1,   1,   1, ...,   1,   1,   1],
        [  1,   1,   1, ...,   1,   1,   1],
        [  2,   2,   2, ...,   2,   2,   2],
        ...,
        [997, 997, 997, ..., 997, 997, 997],
        [998, 998, 998, ..., 998, 998, 998],
        [999, 999, 999, ..., 999, 999, 999]]),
 array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]))

In [193]:
# fix sum x+y+z
XYZ_SUM = 1000

# Define the range of x and y based on the fixed sum constraint
x_vals = np.linspace(1, XYZ_SUM-1, XYZ_SUM, dtype=int)  # 0 < x
y_vals = np.linspace(1, XYZ_SUM-1, XYZ_SUM, dtype=int)  # 0 < y

In [243]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define the sum constraint
XYZ_SUM = 1000

# Define the custom validation function
def valid(a,b):
    c = XYZ_SUM - a - b
    return (sub(V, {x:a, y:b, z:c}) == 1) and eval_f(n,a,b,c) > 0

# Generate ranges for x and y
x_vals = np.linspace(1, XYZ_SUM // 3, 500)  # x should be in (0, XYZ_SUM / 3)
y_vals = np.linspace(1, XYZ_SUM // 2, 500)  # y should be in (0, XYZ_SUM / 2)

# Initialize lists to collect valid points
valid_x = []
valid_y = []
valid_z = []

# Iterate through x and y values to find valid points
for a in x_vals:
    for b in y_vals:
        if valid(a, b):  # Check if the point passes the custom validation function
            valid_x.append(a)
            valid_y.append(b)
            valid_z.append(XYZ_SUM-a-b)

# Scatter plot of valid points
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(valid_x, valid_y, valid_z, c='blue', marker='o', alpha=0.6)

# Label axes
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_title('Scatter Plot of Valid Points in the Region V = {x, y, z : 0 < x < y < z, x + y + z = 1000}')

# Set plot limits
ax.set_xlim(0, XYZ_SUM // 3)
ax.set_ylim(0, XYZ_SUM // 2)
ax.set_zlim(0, XYZ_SUM)

plt.show()

KeyboardInterrupt: 

# LB on h_n(x,y,z) - h_n(y,x,z)

In [288]:
n = 4
H_XYZ = read(H_XYZ_CACHE, min_n=1, max_n=n)
H_YXZ = read(H_YXZ_CACHE, min_n=1, max_n=n)
model = gp.Model(env=gp.Env())
model.reset(1)
gx = model.addVar(lb=1, vtype='I', name='x')
gy = model.addVar(lb=2, vtype='I', name='y')
gz = model.addVar(lb=3, vtype='I', name='z')
SP_TO_GP = {x: gx, y: gy, z: gz}

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Discarded solution information including additional information


In [289]:
model.addConstr(gx <= gy - 1, name='V1')
model.addConstr(gy <= gz - 1, name='V2')

<gurobi.Constr *Awaiting Model Update*>

In [290]:
pos_inds = [add_ind_constrs(*i, var_map=SP_TO_GP) for i in H_XYZ[n][1]]
neg_inds = [add_ind_constrs(*i, var_map=SP_TO_GP) for i in H_YXZ[n][1]]
const = H_XYZ[n][0] - H_YXZ[n][0]

In [291]:
obj = coef(n) * (gp.quicksum([i[0] for i in pos_inds]) - gp.quicksum([i[0] for i in neg_inds]) + const)
model.setObjective(obj, GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 799 columns and 4 nonzeros
Model fingerprint: 0x0e0189e7
Model has 1370 general constraints
Variable types: 0 continuous, 799 integer (796 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-04, 8e-04]
  Bounds range     [1e+00, 3e+00]
  RHS range        [1e+00, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 2e+01]
Presolve added 2073 rows and 1722 columns
Presolve time: 0.02s
Presolved: 2075 rows, 2521 columns, 6037 nonzeros
Presolved model has 1148 SOS constraint(s)
Variable types: 0 continuous, 2521 integer (1370 binary)
Found heuristic solution: objective -0.0007716
Found heuristic solution: objective -0.0015432

Root relaxation: objective -1.118827e-01, 1204 iterations, 0.01 seconds (0.00 work units)

    Node

In [294]:
model.MIPGap

0.0

In [292]:
analyse_result()

Optimal solution found:
x: 7.0, True
y: 8.0, True
z: 23.0, True
single_ind_15*x <= y: -0.0, False
main_ind_(8*x <= z, 7*x < y): -0.0, False
component_ind_8*x <= z: -0.0, False
component_ind_7*x < y: 0.0, False
single_ind_11*x <= y: -0.0, False
main_ind_(12*x <= z, 3*x < y): -0.0, False
component_ind_12*x <= z: 0.0, False
component_ind_3*x < y: -0.0, False
main_ind_(13*x <= 3*y, y < 7*x): -0.0, False
component_ind_13*x <= 3*y: -0.0, False
component_ind_y < 7*x: 1.0, True
main_ind_(7*x <= y + z, y < 7*x, 3*x < y): -0.0, False
component_ind_7*x <= y + z: -0.0, False
component_ind_y < 7*x: 1.0, True
component_ind_3*x < y: -0.0, False
single_ind_5*x <= y: -0.0, False
main_ind_(x <= -y + z, 3*x < y): -0.0, False
component_ind_x <= -y + z: 1.0, True
component_ind_3*x < y: -0.0, False
main_ind_(7*x <= y + z, z < 4*x, 3*x < y): -0.0, False
component_ind_7*x <= y + z: 0.0, False
component_ind_z < 4*x: 1.0, True
component_ind_3*x < y: -0.0, False
main_ind_(4*x <= 3*z, z < 4*x, 3*x < y): -0.0, Fal

In [293]:
model.write('h_4(x,y,z)-h_4(y,x,z).lp')

In [275]:
def eval_dh(n,a,b,c):
    vals = {x:a, y:b, z:c}
    pos_c, pos_rs = H_XYZ[n]
    neg_c, neg_rs = H_YXZ[n]
    return coef(n) * (pos_c - neg_c + sum([sub(r, vals) for r in pos_rs]) - sum([sub(r, vals) for r in neg_rs]))

In [285]:
float(eval_dh(3,5,6,8))

-0.027777777777777776

In [286]:
eval_f(3,5,6,8)

-0.016666666666666677

In [212]:
eval_f(3,4,5,6) + eval_dh(4,7,8,23) + thresh(4)

0.016358024691358018